<a href="https://colab.research.google.com/github/NbtKmy/gc_workshops/blob/main/Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI-Whisper
Whisper ist ein Spracherkennungsmodel, das von OpenAI entwickelt worden ist.
Mit Whisper kann man zum Beispiel eine gesprochene Sprache (Audio-Datei) in einen Text umwandeln (transkribieren).

Im [Github-Repository](https://github.com/openai/whisper) findet man die Code und Beschreibungen.

## Vor der Installation
Vor der Installation überprüfen wir zuerst, ob GPU (Graphicprozessor) für dieses Notebook freigeschaltet ist. Viele KI-Modelle, darunter auch Whisper, braucht GPU.

In [ ]:
!nvidia-smi

Tue Oct 17 12:17:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Wenn der Befehl oben fehlgeschlagen ist, dann muss man die Einstellung ändern.
"Bearbeiten"-"Notebook Einstellungen" klicken. Dann im geöffneten Popup bei "Hardwarebeschleuniger" "GPU" anwählen und Speichern.

Google bietet schnelle Prozessor wie GPU an, aber nur bis zur gewissen Rechnungskapazität und bis zum gewissen Laufzeit des Prozessors. Wenn man noch mehr Rechnungskapazität will, kann man entweder die Angebote von unterschiedlichen Anbietern beziehen oder Graphic Board selber anschaffen und in dem eigenen PC einbauen.

Die [**Science IT**](https://www.zi.uzh.ch/en/teaching-and-research/science-it/computing.html) an der Universität Zürich bietet ebenso die Dienstleistung für solche GPU an.

## Installieren


In [1]:
# Libraries cohere und openai sind notwendig für whisper (Stand: 18.10.2023)
!pip install cohere openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
# Wenn man Whisper von Github installieren will
!pip install git+https://github.com/openai/whisper.git

# Sonst über pip kann man auch whisper installieren
#!pip install -U openai-whisper

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-n2yx0moa
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-n2yx0moa
  Resolved https://github.com/openai/whisper.git to commit b38a1f20f4b23f3f3099af2c3e0ca95627276ddf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230918-py3-none-any.whl size=798405 sha256=0e55d806e5f41fe5e6b48ef6b15bdfe446a105721005f1febfbfec2657744556
  Stored in directory: /tmp/pip-ephem-wheel-cache-mk_05sh8/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


Dann ffmpeg installieren

In [3]:
!sudo apt update && sudo apt install ffmpeg

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,005 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,082 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,349 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com

## Einfache Anwendung durch Commandline

Zuerst ein Audio-File (.flac, .mp3 oder .wav) hochladen!



Bei der Anwendung kann man unterschiedliche Modelle auswählen.

Laut der [offziellen Angabe](https://github.com/openai/whisper#available-models-and-languages) gibt es 5 multi-linguale Modelle.

Je nach der Sprache ergeben sich unterschiedliche Fehlerquoten.

In [ ]:
from google.colab import files

uploaded = files.upload()
uploaded_filename = list(uploaded.keys())[0]
# Unter dem Variabel uploaded_filename ist der Pfad zu dem hochgeladenen File untergebracht

In [ ]:
# Audio anhören
from IPython.display import Audio
Audio(uploaded_filename)

In [ ]:
!whisper $uploaded_filename --model medium

In [ ]:
!whisper $uploaded_filename --task translate

## In einer Python-Code verwenden

In [ ]:
import whisper

model = whisper.load_model('medium')
model.device
result = model.transcribe(uploaded_filename)
print(result['text'])

Mit der Funktion "transcribe()" wird die Audio-Eingabe pro 30 sec. getrennt und Stück für Stück verarbeitet:

>Internally, the transcribe() method reads the entire file and processes the audio with a sliding 30-second window, performing autoregressive sequence-to-sequence predictions on each window.
(aus: [Python usage](https://github.com/openai/whisper#python-usage))

Mit der Funktion "transcribe()" kann man deshalb eine Audio-Eingabe länger als 30 sec. am Stück verarbeiten.
```
model = whisper.load_model([model_size])
audio = whisper.load_audio([filename])
result = model.transcribe(audio, verbose=True, temperature=0.8, language=[lang])
```

Wenn man die andere Funktion "decode()" verwendet, muss die Länge der Audio-Eingabe max. 30 sec. sein. Wenn die Audio-Aufnahme länger als 30 sec. ist, braucht man ein wenig creativ zu sein. Siehe unten...

In [ ]:
import whisper

fileName = 'result_whisper'#@param {type:'string'}
lang = 'de'#@param ['ja', 'en', 'de']
model_size = 'medium'#@param ['tiny', 'base', 'small', 'medium', 'large']
model = whisper.load_model(model_size)


audio = whisper.load_audio(uploaded_filename)
outputTextsArr = []

while audio.size > 0:
    tirmedAudio = whisper.pad_or_trim(audio)
    startIdx = tirmedAudio.size
    audio = audio[startIdx:]

    mel = whisper.log_mel_spectrogram(tirmedAudio).to(model.device)

    options = whisper.DecodingOptions(language=lang, without_timestamps=True)
    result = whisper.decode(model, mel, options)
    outputTextsArr.append(result.text)

outputTexts = ' '.join(outputTextsArr)
print(outputTexts)


with open(f'{fileName}.txt', 'w') as f:
    f.write(outputTexts)

## Ein YouTube-Video mit Untertitel oder Übersetzung versehen

Um ein YouTube-Video herunterzuladen, verwendet man hier [yt-dlp](https://pypi.org/project/yt-dlp/).


In [ ]:
!pip install -U yt-dlp

In [ ]:
from yt_dlp import YoutubeDL

yt_url = 'https://youtu.be/Ifkhj0XtGGM'#@param {type:'string'}
vd_filename = 'test'#@param{type:'string'}
vd_filename += '.mp4'

ydl_opts = {'format': 'best',
            'outtmpl': vd_filename }
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([yt_url])

...Überprüfen, ob das Video tatsächlich heruntergeladen ist...

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open(vd_filename, 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
!pip install -U srt

In [ ]:
import whisper
from datetime import timedelta
from srt import Subtitle
import srt



# Text umbrechen, wenn eine Zeile lang ist　... wahrscheinlich braucht man nicht...
def add_line(line):
    new_line = line
    space_count = new_line.count(' ')
    list_line = list(new_line)
    space_max = 4
    if space_count > space_max :
        space_index = [i for i, x in enumerate(list_line) if x == ' ']
        new_line = line[:space_index[space_max]] + '\n' + line[space_index[space_max]:]

    return new_line

# Übersetzung erstellen
model = whisper.load_model('medium')
result = model.transcribe(vd_filename, task='translate')

segments = result['segments']

subtitles = []

for segment in segments:
    index = segment['id'] + 1
    start = segment['start']
    end = segment['end']
    text = add_line(segment['text'])
    subtitle = Subtitle(
                   index=index,
                   start=timedelta(seconds=timedelta(seconds=start).seconds, microseconds=timedelta(seconds=start).microseconds),
                   end=timedelta(seconds=timedelta(seconds=end).seconds, microseconds=timedelta(seconds=end).microseconds),
                   content=text,
                   proprietary=''
              )
    subtitles.append(subtitle)

# Untertitel speichern
with open('subtitle.srt', mode='w', encoding='utf-8') as f:
    f.write(srt.compose(subtitles))



In [ ]:
!pip install ffmpeg-python

In [ ]:
import ffmpeg

# Das Video mit Untertitel versehen und speichern
video = ffmpeg.input(vd_filename)
audio = video.audio
ffmpeg.concat(video.filter('subtitles', 'subtitle.srt'), audio, v=1, a=1).output('output_vid.mp4').run()

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('output_vid.mp4', 'rb').read()
vid_wSub = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{vid_wSub}" type="video/mp4">
</video>""")